![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

# Spark Setup

In [5]:
import json
import os
import random

license_key = "cabir/keys/6.0.2.spark_nlp_for_healthcare.json"
with open('/home/jovyan/work/shared/' + license_key) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [2]:
# Installing pyspark and spark-nlp
%pip install --upgrade -q pyspark==3.5.0  spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
%pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
%pip install -q spark-nlp-display

In [7]:
import os
import json
import numpy as np
import pandas as pd

# pd.set_option("display.max_colwidth", 100)
# pd.set_option("display.max_colwidth",0)

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

params = {"spark.driver.memory":"58G", # Amount of memory to use for the driver process, i.e. where SparkContext is initialized
          "spark.kryoserializer.buffer.max":"2000M", # Maximum allowable size of Kryo serialization buffer, in MiB unless otherwise specified.
          "spark.driver.maxResultSize":"2000M"} # Limit of total size of serialized results of all partitions for each Spark action (e.g. collect) in bytes.
                                                # Should be at least 1M, or 0 for unlimited.

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)
spark.sparkContext.setLogLevel("ERROR")
print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 6.0.2
Spark NLP_JSL Version : 6.0.2


# Versions

In [1]:
!python --version

Python 3.10.9


In [2]:
!java -version

openjdk version "11.0.17" 2022-10-18
OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu222.04)
OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu222.04, mixed mode, sharing)


In [3]:
%pip show tensorflow

Name: tensorflow
Version: 2.12.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/jovyan/work/shared/venvs/cabir-ds/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, jax, keras, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: 
Note: you may need to restart the kernel to use updated packages.


# Sample text

In [8]:
text = """
(NOTE) Patient Name: John Lee. MR#: 7789201 Location: LERE Date Reported: 2025-05-12 16:30
Specimen #RD23-4897 Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A
Electronically Signed Out By Dr. Smith, Dr. Carter, CT(ASCP) Date Reported: 2025-05-12 16:30
General Hospital Dr. Fan Gabriel 90210 CPT Code(s) A: 88305

General Hospital in New York City Dr. Williams, NYC, NY
(212) 555-7890 Patient Name: John Lee Accession #: GH-556672
Patient ID #: 7789201 Collected: 2025-05-10 Address:
123 Main Street, FALL RIVER
NIAGARA FALLS, NY 14304
Received: 2025-05-10 Reported: 2025-05-12
Soc. Sec. #: XXX-XX-1234 DOB/Age/Sex: 1973 (Age: 52) M
Physician(s): Dr. Jameson. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.
The following special studies were performed at Barstow Heights Christus Southeast, NY – St Elizabeth; New York City.
· Chromosome analysis cytogenetics. (ADDENDUM REPORT TO FOLLOW.)
· Leukemic immunophenotyping flow cytometry.

...., and there is no evidence of dysplasia.
Fr/ap MATERIAL RECEIVED 6 SLIDES LABELED 032-1902, COLLECTED 2025-05-10
SPECIMEN SOURCE: GASTRIC, ILEUM AND RANDOM COLON, BIOPSIES
REFERRING FACILITY: NY
"""

# Pretrained Pipeline

In [8]:
from sparknlp.pretrained import PretrainedPipeline

deid_pipeline = PretrainedPipeline("clinical_deidentification_docwise_benchmark", "en", "clinical/models")

clinical_deidentification_docwise_benchmark download started this may take some time.
Approx size to download 2.3 GB
[OK!]


In [9]:
deid_pipeline.model.stages

[DocumentAssembler_ae0f203deedd,
 InternalDocumentSplitter_cc36578ceda6,
 REGEX_TOKENIZER_2e85686aea12,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NER_CONVERTER_1aef7e9d2de5,
 MedicalNerModel_d92d47622e85,
 MedicalNerModel_32184c1db80b,
 MedicalNerModel_ada39ac0d359,
 NER_CONVERTER_a99db4e6a79d,
 NER_CONVERTER_4a9436714344,
 NER_CONVERTER_ea6433988e18,
 PretrainedZeroShotNER_5f30ab9002f1,
 NER_CONVERTER_c97040caf7b3,
 MedicalNerModel_b8b167ec3114,
 NER_CONVERTER_06db473f3215,
 ContextualEntityRuler_11ff6711ef6b,
 ChunkMergeModel_95d6827691bb,
 CONTEXTUAL-PARSER_bf2a6abaf5fa,
 CONTEXTUAL-PARSER_ff6bad379d91,
 CONTEXTUAL-PARSER_89341cae7221,
 CONTEXTUAL-PARSER_c6b9eded8d31,
 CONTEXTUAL-PARSER_9480c24bd9f8,
 CONTEXTUAL-PARSER_3886bce391c8,
 CONTEXTUAL-PARSER_0bb3fb75cd01,
 ENTITY_EXTRACTOR_6792f2f6e85a,
 ENTITY_EXTRACTOR_74ace4be4f73,
 CONTEXTUAL-PARSER_dfb32adc7555,
 REGEX_MATCHER_5003669d6422,
 CONTEXTUAL-PARSER_746a25662aa6,
 CONTEXTUAL-PARSER_079220479a3d,
 C

## New Stages

In [15]:
document_assembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

splitter = (
            InternalDocumentSplitter()
            .setInputCols("document")
            .setOutputCol("splitter")
            .setSplitMode("recursive")
            .setSplitPatterns(["\s+"])  # Token base
            .setPatternsAreRegex(True)
            .setChunkSize(512)    # 512 Char Lenght
            .setChunkOverlap(50)
            .setEnableSentenceIncrement(True)  # Like sentenceDetector
)

tokenizer = (
    Tokenizer()
    .setInputCols("splitter")
    .setOutputCol("token")
)

### **cpt_parser**

In [16]:
cpt_rule = {
    "entity": "CPT_CODE",
    "ruleScope": "sentence",
    "regex": r"(?:CPT(?: Code\(s\)?|#|:)?\s*:?[\s#]*)?(\b88[0-9]{3}\b)",
    "matchScope": "token"
}

with open('cpt.json', 'w') as f:
    json.dump(cpt_rule, f)

cpt_parser = ContextualParserApproach() \
    .setInputCols(["splitter", "token"]) \
    .setOutputCol("entity_cpt") \
    .setJsonPath("cpt.json") \
    .setCaseSensitive(False) \
    .setPrefixAndSuffixMatch(False)

cpt_parser_pipeline = Pipeline(stages=[
    document_assembler,
    splitter,
    tokenizer,
    cpt_parser
  ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

cpt_parser_model = cpt_parser_pipeline.fit(empty_data)
cpt_parser_model.stages[-1].write().overwrite().save("./parsers/cpt_parser")

cpt_parser = ContextualParserModel.load("parsers/cpt_parser") \
    .setInputCols(["splitter", "token"])\
    .setOutputCol("entity_cpt")

In [17]:
annotations = LightPipeline(cpt_parser_model).annotate(text)

annotations["entity_cpt"]

['88305']

### **specimen_parser**

In [18]:
with open('specimen.json', 'w') as f:
    json.dump({
        "entity": "IDNUM",
        "ruleScope": "sentence",
        "regex": "(?:Specimen(?:\s*(?:ID|Number|Code|#|No\.?)?:?)?\s*)?#?[A-Z]{1,5}[0-9]{2,4}-?[0-9]{3,6}",
        "contextLength": 25,
        "matchScope": "token"
    }, f)

specimen_parser = ContextualParserApproach() \
    .setInputCols(["splitter", "token"]) \
    .setOutputCol("entity_specimen") \
    .setJsonPath("specimen.json") \
    .setCaseSensitive(False) \
    .setPrefixAndSuffixMatch(False)

specimen_parser_pipeline = Pipeline(stages=[
    document_assembler,
    splitter,
    tokenizer,
    specimen_parser
  ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

specimen_parser_model = specimen_parser_pipeline.fit(empty_data)
specimen_parser_model.stages[-1].write().overwrite().save("./parsers/specimen_parser")

specimen_parser = ContextualParserModel.load("./parsers/specimen_parser") \
    .setInputCols(["splitter", "token"])\
    .setOutputCol("entity_specimen")

In [19]:
annotations = LightPipeline(specimen_parser_model).annotate(text)

annotations["entity_specimen"]

['#RD23-4897']

### **IOBTagger**

In [20]:
iobTagger = sparknlp_jsl.annotator.IOBTagger()\
  .setInputCols(["token", "ner_chunk"])\
  .setOutputCol("ner_label")

## **Update Mergers**

In [21]:
merger_input_cols = deid_pipeline.model.stages[35].getInputCols()
#merger_input_cols.remove("entity_zip")
merger_input_cols

['entity_icd10',
 'entity_email',
 'entity_ip_address',
 'entity_age',
 'entity_medicalrecord',
 'entity_ssn',
 'entity_account',
 'entity_vin',
 'entity_date',
 'entity_phone',
 'entity_phone2',
 'entity_country',
 'entity_state',
 'entity_zip',
 'entity_plate',
 'entity_dln',
 'entity_license']

In [22]:
merger_input_cols = deid_pipeline.model.stages[35].getInputCols()

chunk_merge_rulebase = deid_pipeline.model.stages[35]\
      .setInputCols(["entity_cpt", "entity_specimen"] + merger_input_cols)

**black list**

In [23]:
deid_pipeline.model.stages[38]

ChunkMergeModel_5a3f1e608447

In [24]:
deid_pipeline.model.stages[38] = deid_pipeline.model.stages[38]\
                                      .setBlackList(['CPT_CODE'])

## **Stages**

In [25]:
deid_pipeline.model.stages = (
    deid_pipeline.model.stages[:35]
    + [cpt_parser, specimen_parser, chunk_merge_rulebase]
    + deid_pipeline.model.stages[36:]
    + [iobTagger]
)

In [26]:
deid_pipeline.model.stages

[DocumentAssembler_ae0f203deedd,
 InternalDocumentSplitter_cc36578ceda6,
 REGEX_TOKENIZER_2e85686aea12,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NER_CONVERTER_1aef7e9d2de5,
 MedicalNerModel_d92d47622e85,
 MedicalNerModel_32184c1db80b,
 MedicalNerModel_ada39ac0d359,
 NER_CONVERTER_a99db4e6a79d,
 NER_CONVERTER_4a9436714344,
 NER_CONVERTER_ea6433988e18,
 PretrainedZeroShotNER_5f30ab9002f1,
 NER_CONVERTER_c97040caf7b3,
 MedicalNerModel_b8b167ec3114,
 NER_CONVERTER_06db473f3215,
 ContextualEntityRuler_11ff6711ef6b,
 ChunkMergeModel_95d6827691bb,
 CONTEXTUAL-PARSER_bf2a6abaf5fa,
 CONTEXTUAL-PARSER_ff6bad379d91,
 CONTEXTUAL-PARSER_89341cae7221,
 CONTEXTUAL-PARSER_c6b9eded8d31,
 CONTEXTUAL-PARSER_9480c24bd9f8,
 CONTEXTUAL-PARSER_3886bce391c8,
 CONTEXTUAL-PARSER_0bb3fb75cd01,
 ENTITY_EXTRACTOR_6792f2f6e85a,
 ENTITY_EXTRACTOR_74ace4be4f73,
 CONTEXTUAL-PARSER_dfb32adc7555,
 REGEX_MATCHER_5003669d6422,
 CONTEXTUAL-PARSER_746a25662aa6,
 CONTEXTUAL-PARSER_079220479a3d,
 C

In [27]:
empty_result = deid_pipeline.transform(spark.createDataFrame([[""]]).toDF("text"))

deid_pipeline.model.write().overwrite().save("modified_pipeline")

In [20]:
# We are loading the pretrained pipeline using the `from_disk` method.
from sparknlp.pretrained import PretrainedPipeline

modified_pipeline = PretrainedPipeline.from_disk('modified_pipeline')

## Sample Result

In [21]:
samples_df = spark.createDataFrame([[text]]).toDF("text")

result = modified_pipeline.transform(samples_df).cache()

In [22]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(50,truncate=False)

Using CPUs
+----------------------------------+-----+----+---------+----------+
|chunk                             |begin|end |ner_label|confidence|
+----------------------------------+-----+----+---------+----------+
|John Lee                          |22   |29  |NAME     |0.9999912 |
|7789201                           |37   |43  |IDNUM    |0.72      |
|LERE                              |55   |58  |LOCATION |0.8618449 |
|2025-05-12                        |75   |84  |DATE     |NULL      |
|#RD23-4897                        |101  |110 |IDNUM    |0.50      |
|Smith                             |232  |236 |NAME     |0.9992543 |
|Carter                            |243  |248 |NAME     |0.9988757 |
|2025-05-12                        |275  |284 |DATE     |NULL      |
|General Hospital                  |292  |307 |LOCATION |0.9980348 |
|Fan Gabriel                       |313  |323 |NAME     |0.98504204|
|90210                             |325  |329 |IDNUM    |0.5666    |
|General Hospital      

In [23]:
pd.set_option("display.max_colwidth", 1000)
# pd.set_option("display.max_colwidth",0)


result_df = result.selectExpr("text",
                              "mask_entity.result as masked_result",
                              "obfuscated.result as obfuscated_result").toPandas()
result_df

,text,masked_result,obfuscated_result
0,"\n(NOTE) Patient Name: John Lee. MR#: 7789201 Location: LERE Date Reported: 2025-05-12 16:30\nSpecimen #RD23-4897 Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. Smith, Dr. Carter, CT(ASCP) Date Reported: 2025-05-12 16:30\nGeneral Hospital Dr. Fan Gabriel 90210 CPT Code(s) A: 88305\n\nGeneral Hospital in New York City Dr. Williams, NYC, NY\n(212) 555-7890 Patient Name: John Lee Accession #: GH-556672\nPatient ID #: 7789201 Collected: 2025-05-10 Address:\n123 Main Street, FALL RIVER\nNIAGARA FALLS, NY 14304\nReceived: 2025-05-10 Reported: 2025-05-12\nSoc. Sec. #: XXX-XX-1234 DOB/Age/Sex: 1973 (Age: 52) M\nPhysician(s): Dr. Jameson. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed at Barstow Heights Christus Southeast, NY – St Elizabeth; New York City.\n· Chromosome analysis cytogene...","[\n(NOTE) Patient Name: <NAME>. MR#: <IDNUM> Location: <LOCATION> Date Reported: <DATE> 16:30\nSpecimen <IDNUM> Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. <NAME>, Dr. <NAME>, CT(ASCP) Date Reported: <DATE> 16:30\n<LOCATION> Dr. <NAME> <IDNUM> CPT Code(s) A: 88305\n\n<LOCATION> in <LOCATION> City Dr. <NAME>, <LOCATION>, <LOCATION>\n<CONTACT> Patient Name: <NAME> Accession #: <IDNUM>\nPatient ID #: <IDNUM> Collected: <DATE> Address:\n<LOCATION>, <LOCATION>\n<LOCATION>, <LOCATION> <LOCATION>\nReceived: <DATE> Reported: <DATE>\nSoc. Sec. #: <IDNUM> DOB/Age/Sex: <DATE> (Age: <AGE>) M\nPhysician(s): Dr. <NAME>. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed at <LOCATION>, <LOCATION> – <LOCATION>; <LOCATION> City.\n· <LOCATION> analysis cytogenetics. (ADDENDUM REPORT TO FOLLOW.)\n·...","[\n(NOTE) Patient Name: Gillie Allan. MR#: 0074518 Location: 4500 MEMORIAL DRIVE Date Reported: 2025-06-29 16:30\nSpecimen #SA52-9740 Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. Wanna, Dr. Malvin, CT(ASCP) Date Reported: 2025-06-29 16:30\n310 Ellis Street Dr. Marcelo Danes 41581 CPT Code(s) A: 88305\n\n310 Ellis Street in 2000 Boise Ave City Dr. Duwaine, 427 GUY PARK AVE, 16100 SOUTH FREEWAY\n(585) 666-0741 Patient Name: Gillie Allan Accession #: PU-663305\nPatient ID #: 0074518 Collected: 2025-06-27 Address:\n3255 Independence Street, 302 W MCNEESE ST\n4101 NW 89TH BLVD, 16100 SOUTH FREEWAY 59 KOCH AVE\nReceived: 2025-06-27 Reported: 2025-06-29\nSoc. Sec. #: WWW-WW-8529 DOB/Age/Sex: 1974 (Age: 44) M\nPhysician(s): Dr. Marchelle. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed..."


## Dataset

In [31]:
# !wget -q https://github.com/JohnSnowLabs/spark-nlp-workshop/raw/master/tutorials/academic/NER_Benchmarks/ner_cloud_benchmark_text_df.xlsx
# text_df = pd.read_excel("ner_cloud_benchmark_text_df.xlsx")
# #text_df.head(2)
# input_spark_df = spark.createDataFrame(text_df)

In [131]:
# !wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/refs/heads/master/data/ner/eng.train -O eng.train

from sparknlp.training import CoNLL
data_conll = CoNLL(includeDocId=True,explodeSentences=True).readDataset(spark, "./eng.train")
data_conll.show(2)
data_conll.count()


+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|doc_id|                text|            document|            sentence|               token|                 pos|               label|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     X|EU rejects German...|[{document, 0, 47...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     X|     Peter Blackburn|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 4, Pe...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



14041

In [132]:
input_spark_df = data_conll.select("doc_id", "text")
input_spark_df.show(2, truncate=50)

+------+------------------------------------------------+
|doc_id|                                            text|
+------+------------------------------------------------+
|     X|EU rejects German call to boycott British lamb .|
|     X|                                 Peter Blackburn|
+------+------------------------------------------------+
only showing top 2 rows



In [133]:
results = modified_pipeline.transform(input_spark_df)
results.columns

['doc_id',
 'text',
 'document',
 'splitter',
 'token',
 'embeddings',
 'ner_clinical_large',
 'ner_chunk_clinical_large',
 'ner_deid_generic_docwise',
 'ner_deid_docwise_subentity',
 'ner_deid_generic_docwise_merged_conll',
 'ner_chunk_generic_docwise',
 'ner_chunk_subentity_docwise',
 'ner_chunk_merged_docwise',
 'ner_zero_shot',
 'ner_chunk_zero_shot_raw',
 'ner_deid_subentity_docwise_new',
 'ner_chunk_subentity_docwise_new_chunk',
 'ner_chunk_zero_shot',
 'deid_merged_ner_chunk',
 'entity_icd10',
 'entity_ssn',
 'entity_account',
 'entity_dln',
 'entity_plate',
 'entity_vin',
 'entity_license',
 'entity_country',
 'entity_state',
 'entity_age',
 'entity_date',
 'entity_phone',
 'entity_phone2',
 'entity_zip',
 'entity_medicalrecord',
 'entity_email',
 'entity_ip_address',
 'entity_cpt',
 'entity_specimen',
 'deid_merged_ner_rulebased',
 'ner_chunk_raw',
 'ner_chunk_processed',
 'ner_chunk',
 'mask_entity',
 'obfuscated',
 'ner_label']

In [134]:
result_df = results.select('doc_id','text','document','splitter',
                          'token',"embeddings", 'ner_label')

In [135]:
result_df.show(2, truncate=40)

+------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|doc_id|                                    text|                                document|                                splitter|                                   token|                              embeddings|                               ner_label|
+------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|     X|EU rejects German call to boycott Bri...|[{document, 0, 47, EU rejects German ...|[{document, 0, 48, EU rejects German ...|[{token, 0, 1, EU, {sentence -> 0}, [...|[{word_embeddings, 0, 1, EU, {isOOV -...|[{named_entity, 0, 1, 

In [136]:
%%time

n_partitions = 48 

# WRITING THE DATA
result_df.repartition(n_partitions).write.mode("overwrite").format("parquet")\
    .save(f"./data/result_df_{n_partitions}.parquet")

# READING THE DATA
result_df = spark.read \
    .parquet(f"./data/result_df_{n_partitions}.parquet")\
    .repartition(n_partitions)

CPU times: user 122 ms, sys: 29.1 ms, total: 151 ms
Wall time: 12min 16s


# NER Training

In [140]:
result_df.count()

14041

In [141]:
(train_df, test_df) = result_df.randomSplit([0.8, 0.2], seed = 42)

In [142]:
test_df.repartition(n_partitions).write.mode("overwrite").format("parquet")\
    .save(f"./data/test_df.parquet")

## NERDL Graph

We will use `TFGraphBuilder` annotator which can be used to create graphs in the model training pipeline. `TFGraphBuilder` inspects the data and creates the proper graph if a suitable version of TensorFlow is available. The graph is stored in the defined folder and loaded by the `MedicalNerApproach` annotator.

In [ ]:
%pip install -q tensorflow==2.12.0
%pip install -q tensorflow-addons

In [143]:
from sparknlp_jsl.annotator import TFGraphBuilder

graph_folder_path = "medical_ner_graphs"

ner_graph_builder = TFGraphBuilder()\
    .setModelName("ner_dl")\
    .setInputCols(["splitter", "token", "embeddings"]) \
    .setLabelColumn("ner_label")\
    .setGraphFolder(graph_folder_path)\
    .setGraphFile("auto")\
    .setHiddenUnitsNumber(50)\
    .setIsLicensed(True) # False -> if you want to use TFGraphBuilder with NerDLApproach

In [144]:
nerTagger = MedicalNerApproach()\
    .setInputCols(["splitter", "token", "embeddings"])\
    .setLabelColumn("ner_label")\
    .setOutputCol("ner")\
    .setMaxEpochs(30)\
    .setBatchSize(8)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setIncludeConfidence(True)\
    .setOutputLogsPath('ner_logs')\
    .setEarlyStoppingCriterion(0.01)\
    .setEarlyStoppingPatience(5)\
    .setGraphFolder(graph_folder_path)\
    .setUseBestModel(False)\
    #.setTestDataset("./data/test_df.parquet")\
    #.setEnableMemoryOptimizer(True) #>> if you have a limited memory and a large conll file, you can set this True to train batch by batch
    #.setDatasetInfo("NCBI_sample_short dataset") #You can add details regarding the dataset

ner_pipeline = Pipeline(
    stages=[
          ner_graph_builder,
          nerTagger
 ])

In [145]:
%%time
ner_model = ner_pipeline.fit(train_df)

TF Graph Builder configuration:
Model name: ner_dl
Graph folder: medical_ner_graphs
Graph file name: auto
Build params: {'ntags': 13, 'embeddings_dim': 200, 'nchars': 85, 'is_medical': True, 'lstm_size': 50}
ner_dl graph exported to medical_ner_graphs/blstm_13_200_50_85.pb
Training started - total epochs: 30 - lr: 0.001 - batch size: 8 - labels: 13 - chars: 84 - training examples: 11190


Epoch 1/30 started, lr: 0.001, dataset size: 11190


Epoch 1/30 - 26.58s - total training loss: 6000.8135 - avg training loss: 5.372259 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 2.07s
Total validation loss: 923.2025	Avg validation loss: 3.2280
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 663	 115	 195	 0.8521851	 0.77272725	 0.81051344
I-CONTACT	 0	 0	 58	 0.0	 0.0	 0.0
I-AGE	 0	 0	 3	 0.0	 0.0	 0.0
I-IDNUM	 0	 0	 5	 0.0	 0.0	 0.0
B-DATE	 597	 19	 126	 0.96915585	 0.82572615	 0.8917102
I-DATE	 57	 6	 53	 0.9047619	 0.5181818	 0.6589595
I-LOCATION	 221	 67	 535	 0.7673611	 0.29232803	 0.4233716
B-NAME	 751	 233	 243	 0.76321137	 0.7555332	 0.75935286
B-AGE	 18	 13	 95	 0.58064514	 0.15929204	 0.25
B-LOCATION	 1485	 242	 663	 0.85987264	 0.6913408	 0.7664516
B-IDNUM	 0	 0	 26	 0.0	 0.0	 0.0
B-CONTACT	 1	 1	 65	 0.5	 0.015151516	 0.029411767
tp: 3793 fp: 696 fn: 2067 labels: 12
Macro-average	 prec: 0.51643276, rec: 0.33585677, f1: 0.40701526
Micro-average	 prec: 0.8449543, rec: 

Epoch 2/30 - 24.92s - total training loss: 3631.148 - avg training loss: 3.250804 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.77s
Total validation loss: 723.8810	Avg validation loss: 2.5311
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 698	 103	 160	 0.8714107	 0.81351984	 0.8414708
I-CONTACT	 29	 31	 29	 0.48333332	 0.5	 0.4915254
I-AGE	 0	 0	 3	 0.0	 0.0	 0.0
I-IDNUM	 0	 0	 5	 0.0	 0.0	 0.0
B-DATE	 634	 58	 89	 0.91618496	 0.8769018	 0.8961131
I-DATE	 74	 10	 36	 0.88095236	 0.6727273	 0.7628866
I-LOCATION	 348	 132	 408	 0.725	 0.46031746	 0.5631068
B-NAME	 776	 179	 218	 0.81256545	 0.7806841	 0.7963058
B-AGE	 46	 36	 67	 0.5609756	 0.40707964	 0.47179487
B-LOCATION	 1598	 259	 550	 0.86052775	 0.74394786	 0.79800254
B-IDNUM	 0	 1	 26	 0.0	 0.0	 0.0
B-CONTACT	 18	 34	 48	 0.34615386	 0.27272728	 0.30508476
tp: 4221 fp: 843 fn: 1639 labels: 12
Macro-average	 prec: 0.538092, rec: 0.4606588, f1: 0.4963737
Micro-average	 pre

Epoch 3/30 - 25.26s - total training loss: 3031.2466 - avg training loss: 2.7137392 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.78s
Total validation loss: 637.5287	Avg validation loss: 2.2291
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 712	 118	 146	 0.8578313	 0.82983685	 0.8436019
I-CONTACT	 34	 40	 24	 0.45945945	 0.5862069	 0.51515156
I-AGE	 0	 1	 3	 0.0	 0.0	 0.0
I-IDNUM	 0	 0	 5	 0.0	 0.0	 0.0
B-DATE	 657	 92	 66	 0.87716955	 0.9087137	 0.8926631
I-DATE	 86	 10	 24	 0.8958333	 0.7818182	 0.83495146
I-LOCATION	 347	 98	 409	 0.77977526	 0.45899472	 0.5778518
B-NAME	 799	 180	 195	 0.8161389	 0.80382293	 0.80993414
B-AGE	 57	 33	 56	 0.6333333	 0.50442475	 0.56157637
B-LOCATION	 1589	 232	 559	 0.87259746	 0.7397579	 0.80070543
B-IDNUM	 0	 2	 26	 0.0	 0.0	 0.0
B-CONTACT	 19	 28	 47	 0.40425533	 0.28787878	 0.33628318
tp: 4300 fp: 834 fn: 1560 labels: 12
Macro-average	 prec: 0.5496995, rec: 0.4917879, f1: 0.51913357
Mic

...

Epoch 26/30 - 26.06s - total training loss: 1283.1296 - avg training loss: 1.1487284 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.84s
Total validation loss: 516.2067	Avg validation loss: 1.8049
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 743	 107	 115	 0.8741177	 0.8659674	 0.8700234
I-CONTACT	 47	 17	 11	 0.734375	 0.8103448	 0.7704918
I-AGE	 3	 8	 0	 0.27272728	 1.0	 0.42857146
I-IDNUM	 0	 7	 5	 0.0	 0.0	 0.0
B-DATE	 666	 61	 57	 0.9160935	 0.92116183	 0.9186207
I-DATE	 94	 11	 16	 0.8952381	 0.8545455	 0.8744187
I-LOCATION	 473	 252	 283	 0.6524138	 0.6256614	 0.6387575
B-NAME	 827	 171	 167	 0.8286573	 0.831992	 0.8303213
B-AGE	 68	 22	 45	 0.75555557	 0.6017699	 0.6699507
B-LOCATION	 1828	 366	 320	 0.8331814	 0.8510242	 0.8420083
B-IDNUM	 4	 7	 22	 0.36363637	 0.15384616	 0.21621622
B-CONTACT	 36	 28	 30	 0.5625	 0.54545456	 0.5538462
tp: 4789 fp: 1057 fn: 1071 labels: 12
Macro-average	 prec: 0.640708, rec: 0.671814, 

Epoch 27/30 - 27.00s - total training loss: 1269.74 - avg training loss: 1.1367413 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.87s
Total validation loss: 514.8259	Avg validation loss: 1.8001
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 749	 122	 109	 0.8599311	 0.8729604	 0.8663967
I-CONTACT	 48	 25	 10	 0.65753424	 0.82758623	 0.7328244
I-AGE	 0	 1	 3	 0.0	 0.0	 0.0
I-IDNUM	 4	 9	 1	 0.30769232	 0.8	 0.44444445
B-DATE	 668	 63	 55	 0.9138167	 0.9239281	 0.91884464
I-DATE	 91	 6	 19	 0.9381443	 0.8272727	 0.87922704
I-LOCATION	 442	 174	 314	 0.71753246	 0.58465606	 0.6443149
B-NAME	 838	 172	 156	 0.829703	 0.84305835	 0.8363274
B-AGE	 56	 15	 57	 0.7887324	 0.49557522	 0.6086957
B-LOCATION	 1779	 293	 369	 0.8585907	 0.82821226	 0.84312797
B-IDNUM	 8	 15	 18	 0.3478261	 0.30769232	 0.32653064
B-CONTACT	 38	 44	 28	 0.46341464	 0.57575756	 0.5135135
tp: 4721 fp: 939 fn: 1139 labels: 12
Macro-average	 prec: 0.6402432, rec: 

Epoch 28/30 - 25.87s - total training loss: 1245.6982 - avg training loss: 1.1152178 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.92s
Total validation loss: 515.7788	Avg validation loss: 1.8034
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 758	 138	 100	 0.84598213	 0.8834499	 0.8643101
I-CONTACT	 52	 26	 6	 0.6666667	 0.8965517	 0.76470596
I-AGE	 1	 2	 2	 0.33333334	 0.33333334	 0.33333334
I-IDNUM	 4	 17	 1	 0.1904762	 0.8	 0.30769232
B-DATE	 676	 79	 47	 0.8953642	 0.9349931	 0.9147496
I-DATE	 94	 11	 16	 0.8952381	 0.8545455	 0.8744187
I-LOCATION	 404	 132	 352	 0.75373137	 0.5343915	 0.625387
B-NAME	 819	 161	 175	 0.8357143	 0.8239437	 0.8297872
B-AGE	 63	 14	 50	 0.8181818	 0.5575221	 0.6631579
B-LOCATION	 1763	 279	 385	 0.8633692	 0.8207635	 0.8415274
B-IDNUM	 12	 19	 14	 0.38709676	 0.46153846	 0.42105266
B-CONTACT	 40	 43	 26	 0.48192772	 0.6060606	 0.53691274
tp: 4686 fp: 921 fn: 1174 labels: 12
Macro-average	 prec

Epoch 29/30 - 26.33s - total training loss: 1222.2452 - avg training loss: 1.0942214 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.82s
Total validation loss: 524.0858	Avg validation loss: 1.8325
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 746	 117	 112	 0.86442643	 0.86946386	 0.8669379
I-CONTACT	 49	 30	 9	 0.62025315	 0.8448276	 0.71532845
I-AGE	 0	 2	 3	 0.0	 0.0	 0.0
I-IDNUM	 0	 4	 5	 0.0	 0.0	 0.0
B-DATE	 668	 55	 55	 0.9239281	 0.9239281	 0.9239281
I-DATE	 91	 9	 19	 0.91	 0.8272727	 0.8666666
I-LOCATION	 432	 167	 324	 0.721202	 0.5714286	 0.6376384
B-NAME	 822	 160	 172	 0.8370672	 0.82696176	 0.8319838
B-AGE	 66	 21	 47	 0.7586207	 0.5840708	 0.66
B-LOCATION	 1743	 260	 405	 0.87019473	 0.8114525	 0.8397976
B-IDNUM	 5	 23	 21	 0.17857143	 0.1923077	 0.1851852
B-CONTACT	 32	 44	 34	 0.42105263	 0.4848485	 0.45070425
tp: 4654 fp: 892 fn: 1206 labels: 12
Macro-average	 prec: 0.5921097, rec: 0.57804686, f1: 0.5849938
Mi

Epoch 30/30 - 25.80s - total training loss: 1198.5831 - avg training loss: 1.0730377 - batches: 1117
Quality on validation dataset (20.0%), validation examples = 2238
time to finish evaluation: 1.83s
Total validation loss: 529.5698	Avg validation loss: 1.8516
label	 tp	 fp	 fn	 prec	 rec	 f1
I-NAME	 751	 111	 107	 0.8712297	 0.87529135	 0.8732558
I-CONTACT	 43	 20	 15	 0.6825397	 0.7413793	 0.7107439
I-AGE	 1	 6	 2	 0.14285715	 0.33333334	 0.2
I-IDNUM	 0	 4	 5	 0.0	 0.0	 0.0
B-DATE	 670	 60	 53	 0.91780823	 0.92669433	 0.9222299
I-DATE	 89	 10	 21	 0.8989899	 0.8090909	 0.85167474
I-LOCATION	 436	 189	 320	 0.6976	 0.5767196	 0.63142645
B-NAME	 841	 167	 153	 0.8343254	 0.8460765	 0.84015983
B-AGE	 75	 26	 38	 0.7425743	 0.6637168	 0.7009346
B-LOCATION	 1796	 337	 352	 0.84200656	 0.8361266	 0.8390563
B-IDNUM	 3	 11	 23	 0.21428572	 0.115384616	 0.14999999
B-CONTACT	 31	 44	 35	 0.41333333	 0.46969697	 0.4397163
tp: 4736 fp: 985 fn: 1124 labels: 12
Macro-average	 prec: 0.6047959, rec: 

In [146]:
ner_model.stages[-1].getTrainingClassDistribution()

{'I-NAME': 4341, 'I-CONTACT': 205, 'I-AGE': 36, 'I-IDNUM': 63, 'B-DATE': 3513, 'I-DATE': 490, 'I-LOCATION': 3780, 'B-NAME': 5133, 'B-AGE': 589, 'B-LOCATION': 10578, 'B-IDNUM': 160, 'O': 136507, 'B-CONTACT': 311}

In [150]:
ner_model.stages[-1].write().overwrite().save('models/new_NER_model')

In [ ]:
import os
log_file= os.listdir("ner_logs")[0]

with open (f"./ner_logs/{log_file}") as f:
    print(f.read())

## Evaluate your model

In [147]:
pred_df = ner_model.stages[-1].transform(test_df).cache()

In [148]:
pred_df.show()

+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|doc_id|                text|            document|            splitter|               token|          embeddings|           ner_label|                 ner|
+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|     X|" Bankruptcy is a...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 0, ",...|[{word_embeddings...|[{named_entity, 0...|[{named_entity, 0...|
|     X|" I have my own c...|[{document, 0, 57...|[{document, 0, 58...|[{token, 0, 0, ",...|[{word_embeddings...|[{named_entity, 0...|[{named_entity, 0...|
|     X|" Indian fisherme...|[{document, 0, 12...|[{document, 0, 12...|[{token, 0, 0, ",...|[{word_embeddings...|[{named_entity, 0...|[{named_entity, 0...|
|     X|" The first half ...|[{document, 0, 20...|[{document, 0,

In [151]:
from pyspark.sql import functions as F

pred_token_df = pred_df.select(F.explode(F.arrays_zip(pred_df.ner_label.metadata,
                                                  pred_df.ner_label.begin,
                                                  pred_df.ner_label.end,
                                                  pred_df.ner_label.result,
                                                  pred_df.ner.result)).alias("cols")) \
          .select(F.expr("cols['0']['word']").alias("token"),
                  F.expr("cols['1']").alias("begin"),
                  F.expr("cols['2']").alias("end"),
                  F.expr("cols['3']").alias("gtruth"),
                  F.expr("cols['4']").alias("prediction"))\
          .toPandas()

pred_token_df

,token,begin,end,gtruth,prediction
0,"""",0,0,O,O
1,Bankruptcy,2,11,B-NAME,B-NAME
2,is,13,14,O,O
3,always,16,21,O,O
4,possible,23,30,O,O
...,...,...,...,...,...
40252,Wednesday,21,29,B-DATE,B-DATE
40253,(,31,31,O,O
40254,prefix,33,38,O,O
40255,number,40,45,O,O


In [153]:
from sparknlp_jsl.eval import NerDLMetrics
import pyspark.sql.functions as F

evaler = NerDLMetrics(mode="full_chunk")

eval_result = evaler.computeMetricsFromDF(pred_df.select("ner_label","ner"), 
                                          prediction_col="ner", 
                                          label_col="ner_label", 
                                          drop_o = True, case_sensitive = True).cache()

eval_result.withColumn("precision", F.round(eval_result["precision"],4))\
           .withColumn("recall", F.round(eval_result["recall"],4))\
           .withColumn("f1", F.round(eval_result["f1"],4)).show(100)

print(eval_result.selectExpr("avg(f1) as macro").show())
print (eval_result.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+--------+------+-----+-----+------+---------+------+------+
|  entity|    tp|   fp|   fn| total|precision|recall|    f1|
+--------+------+-----+-----+------+---------+------+------+
| CONTACT|  42.0| 30.0| 29.0|  71.0|   0.5833|0.5915|0.5874|
|    NAME|1149.0|143.0|162.0|1311.0|   0.8893|0.8764|0.8828|
|    DATE| 809.0| 29.0| 68.0| 877.0|   0.9654|0.9225|0.9434|
|   IDNUM|  18.0|  8.0| 20.0|  38.0|   0.6923|0.4737|0.5625|
|LOCATION|2260.0|310.0|325.0|2585.0|   0.8794|0.8743|0.8768|
|     AGE| 133.0| 32.0| 23.0| 156.0|   0.8061|0.8526|0.8287|
+--------+------+-----+-----+------+---------+------+------+

+-----------------+
|            macro|
+-----------------+
|0.780276564367539|
+-----------------+

None
+------------------+
|             micro|
+------------------+
|0.8820389851678987|
+------------------+

None


In [154]:
evaler = NerDLMetrics(mode="partial_chunk_per_token")
eval_result_partial = evaler.computeMetricsFromDF(pred_df.select("ner_label","ner"), prediction_col="ner", label_col="ner_label", drop_o = True, case_sensitive = True).cache()

eval_result_partial.withColumn("precision", F.round(eval_result_partial["precision"],4))\
           .withColumn("recall", F.round(eval_result_partial["recall"],4))\
           .withColumn("f1", F.round(eval_result_partial["f1"],4)).sort("entity").show(100)
df_partial=eval_result_partial.toPandas()
print("partial_chunk_per_token")
print(eval_result_partial.selectExpr("avg(f1) as macro").show())
print (eval_result_partial.selectExpr("sum(f1*total) as sumprod","sum(total) as sumtotal").selectExpr("sumprod/sumtotal as micro").show())

+--------+------+-----+-----+------+---------+------+------+
|  entity|    tp|   fp|   fn| total|precision|recall|    f1|
+--------+------+-----+-----+------+---------+------+------+
|     AGE| 138.0| 31.0| 23.0| 161.0|   0.8166|0.8571|0.8364|
| CONTACT| 106.0| 36.0| 24.0| 130.0|   0.7465|0.8154|0.7794|
|    DATE| 934.0| 24.0| 66.0|1000.0|   0.9749| 0.934| 0.954|
|   IDNUM|  25.0| 11.0| 22.0|  47.0|   0.6944|0.5319|0.6024|
|LOCATION|3140.0|330.0|392.0|3532.0|   0.9049| 0.889|0.8969|
|    NAME|2181.0|171.0|185.0|2366.0|   0.9273|0.9218|0.9245|
+--------+------+-----+-----+------+---------+------+------+

partial_chunk_per_token
+------------------+
|             macro|
+------------------+
|0.8322751118663967|
+------------------+

None
+------------------+
|             micro|
+------------------+
|0.9084578982865754|
+------------------+

None


# Create New Pipeline

In [180]:
# We are loading the pretrained pipeline using the `from_disk` method.
from sparknlp.pretrained import PretrainedPipeline

modified_pipeline = PretrainedPipeline.from_disk('modified_pipeline')

In [181]:
modified_pipeline.model.stages

[DocumentAssembler_ae0f203deedd,
 InternalDocumentSplitter_cc36578ceda6,
 REGEX_TOKENIZER_2e85686aea12,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NER_CONVERTER_1aef7e9d2de5,
 MedicalNerModel_d92d47622e85,
 MedicalNerModel_32184c1db80b,
 MedicalNerModel_ada39ac0d359,
 NER_CONVERTER_a99db4e6a79d,
 NER_CONVERTER_4a9436714344,
 NER_CONVERTER_ea6433988e18,
 PretrainedZeroShotNER_5f30ab9002f1,
 NER_CONVERTER_c97040caf7b3,
 MedicalNerModel_b8b167ec3114,
 NER_CONVERTER_06db473f3215,
 ContextualEntityRuler_11ff6711ef6b,
 ChunkMergeModel_95d6827691bb,
 CONTEXTUAL-PARSER_bf2a6abaf5fa,
 CONTEXTUAL-PARSER_ff6bad379d91,
 CONTEXTUAL-PARSER_89341cae7221,
 CONTEXTUAL-PARSER_c6b9eded8d31,
 CONTEXTUAL-PARSER_9480c24bd9f8,
 CONTEXTUAL-PARSER_3886bce391c8,
 CONTEXTUAL-PARSER_0bb3fb75cd01,
 ENTITY_EXTRACTOR_6792f2f6e85a,
 ENTITY_EXTRACTOR_74ace4be4f73,
 CONTEXTUAL-PARSER_dfb32adc7555,
 REGEX_MATCHER_5003669d6422,
 CONTEXTUAL-PARSER_746a25662aa6,
 CONTEXTUAL-PARSER_079220479a3d,
 C

## New Stages

In [179]:
ner_deid_new = MedicalNerModel.load("models/new_NER_model")\
    .setInputCols(["splitter", "token", "embeddings"])\
    .setOutputCol("ner_deid_new")

ner_deid_new_converter = NerConverter()\
      .setInputCols(["splitter", "token", "ner_deid_new"])\
      .setOutputCol("ner_chunk_new")

ner_deid = MedicalNerModel.pretrained("ner_deid_subentity_docwise", "en", "clinical/models")  \
      .setInputCols(["splitter", "token", "embeddings"]) \
      .setOutputCol("ner_deid_subentity_docwise")

ner_deid_converter = NerConverter()\
      .setInputCols(["splitter", "token", "ner_deid_subentity_docwise"])\
      .setOutputCol("ner_chunk_subentity_docwise")

chunk_merge_ner = ChunkMergeModel()\
    .setInputCols("ner_chunk_new", # New Trained Model
                  "ner_chunk_subentity_docwise")\
    .setOutputCol("deid_merged_ner_chunk")\
    .setOrderingFeatures(["ChunkLength","ChunkBegin"])\
    .setMergeOverlapping(True)\
    .setResetSentenceIndices(True)


ner_deid_subentity_docwise download started this may take some time.
[OK!]


## **Update Stages**

In [182]:
modified_pipeline.model.stages = (
    modified_pipeline.model.stages[:4]
    + [ner_deid_new, 
       ner_deid_new_converter,
       ner_deid, 
       ner_deid_converter,
       chunk_merge_ner]
    + modified_pipeline.model.stages[18:]

)

In [183]:
modified_pipeline.model.stages

[DocumentAssembler_ae0f203deedd,
 InternalDocumentSplitter_cc36578ceda6,
 REGEX_TOKENIZER_2e85686aea12,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_f3c446a6c387,
 NerConverter_e2443d1f19ec,
 MedicalNerModel_32184c1db80b,
 NerConverter_0e94e14fb143,
 ChunkMergeModel_15094e842f96,
 CONTEXTUAL-PARSER_bf2a6abaf5fa,
 CONTEXTUAL-PARSER_ff6bad379d91,
 CONTEXTUAL-PARSER_89341cae7221,
 CONTEXTUAL-PARSER_c6b9eded8d31,
 CONTEXTUAL-PARSER_9480c24bd9f8,
 CONTEXTUAL-PARSER_3886bce391c8,
 CONTEXTUAL-PARSER_0bb3fb75cd01,
 ENTITY_EXTRACTOR_6792f2f6e85a,
 ENTITY_EXTRACTOR_74ace4be4f73,
 CONTEXTUAL-PARSER_dfb32adc7555,
 REGEX_MATCHER_5003669d6422,
 CONTEXTUAL-PARSER_746a25662aa6,
 CONTEXTUAL-PARSER_079220479a3d,
 CONTEXTUAL-PARSER_f8b8f9aafb9f,
 CONTEXTUAL-PARSER_7f824493eafc,
 REGEX_MATCHER_26934077fe57,
 REGEX_MATCHER_5fe3de8b5a4e,
 CONTEXTUAL-PARSER_1543b24b4890,
 CONTEXTUAL-PARSER_980d396bec6c,
 MERGE_ddff59e8b14a,
 ChunkMergeModel_50feb5f97568,
 ContextualEntityRuler_08eeaa89c938,
 ChunkMe

In [185]:
empty_result = modified_pipeline.transform(spark.createDataFrame([[""]]).toDF("text"))

modified_pipeline.model.write().overwrite().save("new_pipeline")

In [11]:
from sparknlp.pretrained import PretrainedPipeline

new_pipeline = PretrainedPipeline.from_disk('new_pipeline')

## Sample Result

In [10]:
samples_df = spark.createDataFrame([[text]]).toDF("text")

result = new_pipeline.transform(samples_df).cache()

In [12]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result,
                                     result.ner_chunk.begin,
                                     result.ner_chunk.end,
                                     result.ner_chunk.metadata)).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']").alias("begin"),
              F.expr("cols['2']").alias("end"),
              F.expr("cols['3']['entity']").alias("ner_label"),
              F.expr("cols['3']['confidence']").alias("confidence")).show(50,truncate=False)

+----------------------------------+-----+----+---------+----------+
|chunk                             |begin|end |ner_label|confidence|
+----------------------------------+-----+----+---------+----------+
|John Lee                          |22   |29  |NAME     |0.83455   |
|7789201                           |37   |43  |IDNUM    |0.72      |
|2025-05-12                        |75   |84  |DATE     |NULL      |
|#RD23-4897                        |101  |110 |IDNUM    |0.50      |
|Smith                             |232  |236 |NAME     |0.6658    |
|Carter                            |243  |248 |NAME     |0.9997    |
|2025-05-12                        |275  |284 |DATE     |NULL      |
|Fan Gabriel                       |313  |323 |NAME     |0.7408    |
|90210                             |325  |329 |IDNUM    |0.3806    |
|New York                          |373  |380 |LOCATION |NULL      |
|Williams                          |391  |398 |LOCATION |0.3922    |
|NYC                              

In [19]:
pd.set_option("display.max_colwidth", 1000)
# pd.set_option("display.max_colwidth",0)


result_df = result.selectExpr("text","mask_entity.result as masked_result","obfuscated.result as obfuscated_result").toPandas()
result_df

,text,masked_result,obfuscated_result
0,"\n(NOTE) Patient Name: John Lee. MR#: 7789201 Location: LERE Date Reported: 2025-05-12 16:30\nSpecimen #RD23-4897 Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. Smith, Dr. Carter, CT(ASCP) Date Reported: 2025-05-12 16:30\nGeneral Hospital Dr. Fan Gabriel 90210 CPT Code(s) A: 88305\n\nGeneral Hospital in New York City Dr. Williams, NYC, NY\n(212) 555-7890 Patient Name: John Lee Accession #: GH-556672\nPatient ID #: 7789201 Collected: 2025-05-10 Address:\n123 Main Street, FALL RIVER\nNIAGARA FALLS, NY 14304\nReceived: 2025-05-10 Reported: 2025-05-12\nSoc. Sec. #: XXX-XX-1234 DOB/Age/Sex: 1973 (Age: 52) M\nPhysician(s): Dr. Jameson. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed at Barstow Heights Christus Southeast, NY – St Elizabeth; New York City.\n· Chromosome analysis cytogene...","[\n(NOTE) Patient Name: <NAME>. MR#: <IDNUM> Location: LERE Date Reported: <DATE> 16:30\nSpecimen <IDNUM> Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. <NAME>, Dr. <NAME>, CT(ASCP) Date Reported: <DATE> 16:30\nGeneral Hospital Dr. <NAME> <IDNUM> CPT Code(s) A: 88305\n\nGeneral Hospital in <LOCATION> City Dr. <LOCATION>, <LOCATION>, <LOCATION>\n<CONTACT> Patient Name: <NAME> Accession #: <IDNUM>\nPatient ID #: <IDNUM> Collected: <DATE> Address:\n<LOCATION>, FALL <LOCATION>, <LOCATION> <LOCATION>\nReceived: <DATE> Reported: <DATE>\nSoc. Sec. #: XXX-XX-1234 DOB/Age/Sex: <DATE> (Age: <AGE>) <NAME>): Dr. <NAME>. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed at <LOCATION>, <LOCATION> – <LOCATION>; <LOCATION> City.\n· Chromosome analysis cytogenetics. (ADDENDUM REPORT TO FOLLOW.)\n· ...","[\n(NOTE) Patient Name: Gillie Allan. MR#: 0074518 Location: LERE Date Reported: 2025-06-29 16:30\nSpecimen #SA52-9740 Clinical History: None Given. CLINICAL INFORMATION: Date of Last Menstrual Period: N/A\nElectronically Signed Out By Dr. Wanna, Dr. Malvin, CT(ASCP) Date Reported: 2025-06-29 16:30\nGeneral Hospital Dr. Marcelo Danes 41581 CPT Code(s) A: 88305\n\nGeneral Hospital in 2000 Boise Ave City Dr. 2601 Fox Run Parkway, 427 GUY PARK AVE, 16100 SOUTH FREEWAY\n(585) 666-0741 Patient Name: Gillie Allan Accession #: PU-663305\nPatient ID #: 0074518 Collected: 2025-06-27 Address:\n3255 Independence Street, FALL 401 BICENTENNIAL WAY, 16100 SOUTH FREEWAY 59 KOCH AVE\nReceived: 2025-06-27 Reported: 2025-06-29\nSoc. Sec. #: XXX-XX-1234 DOB/Age/Sex: 1974 (Age: 44) TERETHA Sol): Dr. Marchelle. Peripheral sequestration, i.e. splenomegaly or hepatomegaly should be excluded to be sure if peripheral sequestration is not present.\nThe following special studies were performed at 103 North S..."
